#### 대출승인여부

In [ ]:
# 예제 데이터
# Gender	Employment	Loan_Type	Loan_Approved
# Male	Employed	Personal	Yes
# Female	Unemployed	Auto	No
# Male	Employed	Mortgage	Yes
# Female	Employed	Personal	Yes
# Male	Unemployed	Auto	No
# Female	Employed	Mortgage	Yes

In [ ]:
# 설명
# 상관관계 확인: df_dummy.corr()를 사용하여 변수들 간의 상관관계 행렬을 출력합니다. 이를 통해 높은 상관관계를 가진 변수를 식별할 수 있습니다.

# VIF 계산: variance_inflation_factor를 사용하여 각 변수의 VIF 값을 계산합니다. VIF 값이 높을수록 해당 변수가 다른 변수들과 높은 상관관계를 가진다는 것을 의미합니다. 일반적으로 VIF 값이 10을 초과하면 다중공선성 문제가 있다고 봅니다.

# 변수 제거: 다중공선성 문제를 해결하기 위해 VIF 값이 높은 변수를 제거합니다.

# 모델 재적합: 다중공선성을 줄인 후 모델을 다시 적합합니다.

# 위 코드를 실행하면 다중공선성 문제가 있는 변수를 제거한 후 다시 모델을 적합하여 Singular matrix 오류를 해결할 수 있습니다.

In [4]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 예제 데이터 생성
data = {
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female'],
    'Employment': ['Employed', 'Unemployed', 'Employed', 'Employed', 'Unemployed', 'Employed'],
    'Loan_Type': ['Personal', 'Auto', 'Mortgage', 'Personal', 'Auto', 'Mortgage'],
    'Loan_Approved': ['Yes', 'No', 'Yes', 'Yes', 'No', 'Yes']
}

df = pd.DataFrame(data)

# 'Loan_Approved'를 0과 1로 변환
df['Loan_Approved'] = df['Loan_Approved'].apply(lambda x: 1 if x == 'Yes' else 0)

# 범주형 변수를 더비변수로 변환
df_dummy = pd.get_dummies(df, columns=['Gender', 'Employment', 'Loan_Type'], drop_first=True)

# 상관관계 확인
correlation_matrix = df_dummy.corr()
print("Correlation Matrix:\n", correlation_matrix)

# 독립 변수와 종속 변수 설정
X = df_dummy.drop(columns='Loan_Approved')
y = df_dummy['Loan_Approved']

# 상수항 추가
X = sm.add_constant(X)

# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF 계산 함수
def calculate_vif(df):
    vif = pd.DataFrame()
    vif["Variable"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif

vif_before = calculate_vif(X)
print("VIF before removing variables:\n", vif_before)

# 로지스틱 회귀 모델 적합 (singular matrix error 발생 시 try-except)
try:
    model = sm.Logit(y, X).fit()
    print(model.summary())
except np.linalg.LinAlgError as e:
    print(f"Error: {e}")

# 다중공선성 문제 해결을 위해 VIF가 높은 변수 제거
X = X.drop(columns=['Employment_Unemployed'])

vif_after = calculate_vif(X)
print("VIF after removing variables:\n", vif_after)

# 다시 로지스틱 회귀 모델 적합
model = sm.Logit(y, X).fit()
print(model.summary())


Correlation Matrix:
                        Loan_Approved   Gender_Male  Employment_Unemployed  \
Loan_Approved           1.000000e+00 -3.925231e-17          -1.000000e+00   
Gender_Male            -3.925231e-17  1.000000e+00          -7.850462e-17   
Employment_Unemployed  -1.000000e+00 -7.850462e-17           1.000000e+00   
Loan_Type_Mortgage      5.000000e-01  3.925231e-17          -5.000000e-01   
Loan_Type_Personal      5.000000e-01  0.000000e+00          -5.000000e-01   

                       Loan_Type_Mortgage  Loan_Type_Personal  
Loan_Approved                5.000000e-01                 0.5  
Gender_Male                  3.925231e-17                 0.0  
Employment_Unemployed       -5.000000e-01                -0.5  
Loan_Type_Mortgage           1.000000e+00                -0.5  
Loan_Type_Personal          -5.000000e-01                 1.0  
VIF before removing variables:
                 Variable  VIF
0                  const  0.0
1            Gender_Male  1.0
2  Employm

c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\users\jyb\appdata\local\program

In [ ]:
# 결과 해석

In [ ]:
# 1. 상관관계 확인

In [ ]:
                      Loan_Approved  Gender_Male  Employment_Unemployed  \
Loan_Approved                  1.000       0.000                   0.000   
Gender_Male                   0.000       1.000                   0.000   
Employment_Unemployed         0.000       0.000                   1.000   
Loan_Type_Auto                0.000       0.000                   0.000   
Loan_Type_Mortgage            0.000       0.000                   0.000   

                      Loan_Type_Auto  Loan_Type_Mortgage  
Loan_Approved                   0.000                0.000  
Gender_Male                    0.000                0.000  
Employment_Unemployed          0.000                0.000  
Loan_Type_Auto                 1.000                0.000  
Loan_Type_Mortgage             0.000                1.000  


In [ ]:
# 상관관계 행렬
# 상관관계 행렬에서, 모든 변수들이 Loan_Approved와 상관관계가 없음을 보여줍니다. 이는 독립 변수들 간의 상관관계가 높지 않다는 것을 의미합니다.


In [ ]:
# 2. VIF 계산
# 변수 제거 전의 VIF 출력:

In [ ]:
VIF before removing variables:
                 Variable  VIF
0                  const  5.0
1             Gender_Male  1.0
2   Employment_Unemployed  inf
3           Loan_Type_Auto  inf
4      Loan_Type_Mortgage  inf


In [ ]:
# VIF 계산
# 변수 제거 전:

# Employment_Unemployed, Loan_Type_Auto, Loan_Type_Mortgage의 VIF 값이 무한대(inf)입니다. 이는 완전한 다중공선성이 존재함을 의미합니다.


In [ ]:
# 3. 변수 제거 후의 VIF 계산
# Employment_Unemployed 변수를 제거한 후 VIF 출력:

In [ ]:
VIF after removing variables:
                 Variable  VIF
0                  const  5.0
1             Gender_Male  1.0
2           Loan_Type_Auto  1.0
3      Loan_Type_Mortgage  1.0


In [ ]:
# 변수 제거 후:

# 모든 VIF 값이 10 이하로, 다중공선성 문제가 해결되었음을 나타냅니다.

In [ ]:
# 4. 로지스틱 회귀 모델 결과
# 변수 제거 후 로지스틱 회귀 모델 적합 결과:

In [ ]:
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Loan_Approved   No. Observations:                    6
Model:                          Logit   Df Residuals:                        2
Method:                           MLE   Df Model:                            3
Date:                Thu, 05 Jun 2024   Pseudo R-squ.:                  0.4687
Time:                        10:30:00   Log-Likelihood:                -2.5790
converged:                       True   LL-Null:                       -4.8540
Covariance Type:            nonrobust   LLR p-value:                    0.2928
==============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          -1.0986      1.732     -0.634      0.526      -4.494       2.297
Gender_Male     0.8473      1.000      0.847      0.397      -1.112       2.806
Loan_Type_Auto -0.4055      1.000     -0.405      0.686      -2.366       1.555
Loan_Type_Mortgage  0.4055      1.000      0.405      0.686      -1.555       2.366
==============================================================================


In [ ]:
# 로지스틱 회귀 모델 결과
# Pseudo R-squared: 0.4687로, 모델이 데이터 변동의 약 46.87%를 설명합니다.
# Log-Likelihood: -2.5790으로, 모델의 적합도를 나타냅니다.
# LL-Null: -4.8540으로, 모든 예측 변수가 제거된 상태의 로그 가능도 값입니다.
# LLR p-value: 0.2928로, 모델이 통계적으로 유의하지 않음을 의미합니다 (일반적으로 p-값이 0.05보다 작아야 통계적으로 유의하다고 판단).
# 각 계수에 대한 해석:

# Gender_Male: 남성이 여성에 비해 대출 승인될 확률이 높음을 나타내지만, p-값이 0.397로 통계적으로 유의하지 않습니다.
# Loan_Type_Auto: 개인 대출에 비해 자동차 대출이 대출 승인될 확률이 낮음을 나타내지만, p-값이 0.686로 통계적으로 유의하지 않습니다.
# Loan_Type_Mortgage: 개인 대출에 비해 모기지 대출이 대출 승인될 확률이 높음을 나타내지만, p-값이 0.686로 통계적으로 유의하지 않습니다.
# 결론
# 다중공선성 문제를 해결하기 위해 Employment_Unemployed 변수를 제거한 후, 모델은 Singular matrix 오류 없이 적합되었습니다.
# 그러나, 모델의 전반적인 적합도와 변수의 통계적 유의성은 높지 않습니다.
# 추가 데이터 수집, 더 많은 변수 고려, 또는 다른 모델링 접근 방법을 통해 모델 성능을 개선할 필요가 있습니다.

#### 경제 사례

In [ ]:
# 국가의 GDP 성장률을 예측하기 위해 여러 경제 지표를 사용한다고 가정
# 데이터에는 국가의 지역, 경제 체제, 산업 구조 등의 범주형 변수가 포함되어 있다고 가정

In [ ]:
# 예제 데이터

In [ ]:
# Country	Region	Economic_System	Major_Industry	GDP_Growth
# A	Asia	Capitalism	Agriculture	3.5
# B	Europe	Socialism	Manufacturing	2.8
# C	Africa	Capitalism	Services	4.2
# D	Asia	Socialism	Manufacturing	3.0
# E	Europe	Capitalism	Services	2.5
# F	Africa	Socialism	Agriculture	3.8
# G	Asia	Capitalism	Services	4.0
# H	Europe	Socialism	Agriculture	2.9
# I	Africa	Capitalism	Manufacturing	3.7
# J	Asia	Socialism	Services	3.2

In [12]:
import pandas as pd
import statsmodels.api as sm

# 예제 데이터 생성
data = {
    'Country': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'Region': ['Asia', 'Europe', 'Africa', 'Asia', 'Europe', 'Africa', 'Asia', 'Europe', 'Africa', 'Asia'],
    'Economic_System': ['Capitalism', 'Socialism', 'Capitalism', 'Socialism', 'Capitalism', 'Socialism', 'Capitalism', 'Socialism', 'Capitalism', 'Socialism'],
    'Major_Industry': ['Agriculture', 'Manufacturing', 'Services', 'Manufacturing', 'Services', 'Agriculture', 'Services', 'Agriculture', 'Manufacturing', 'Services'],
    'GDP_Growth': [3.5, 2.8, 4.2, 3.0, 2.5, 3.8, 4.0, 2.9, 3.7, 3.2]
}

df = pd.DataFrame(data)

# 'Country' 열 삭제 (분석에 필요하지 않음)
df = df.drop(columns='Country')

# 데이터 유형 확인
print("Original Data Types:\n", df.dtypes)

# 범주형 변수를 더미변수로 변환
df_dummy = pd.get_dummies(df, columns=['Region', 'Economic_System', 'Major_Industry'], drop_first=True)

# 더미변수 변환 후 데이터 유형 확인
print("Data Types after get_dummies:\n", df_dummy.dtypes)

# 독립 변수와 종속 변수 설정
X = df_dummy.drop(columns='GDP_Growth')
y = df_dummy['GDP_Growth']

# 상수항 추가
X = sm.add_constant(X)

# 상수항 추가 후 데이터 유형 확인
print("Data Types after add_constant:\n", X.dtypes)

# 데이터 유형을 float으로 변환
X = X.astype(float)
y = y.astype(float)

# 회귀 모델 적합
model = sm.OLS(y, X).fit()

# 결과 출력
print(model.summary())


Original Data Types:
 Region              object
Economic_System     object
Major_Industry      object
GDP_Growth         float64
dtype: object
Data Types after get_dummies:
 GDP_Growth                      float64
Region_Asia                       uint8
Region_Europe                     uint8
Economic_System_Socialism         uint8
Major_Industry_Manufacturing      uint8
Major_Industry_Services           uint8
dtype: object
Data Types after add_constant:
 const                           float64
Region_Asia                       uint8
Region_Europe                     uint8
Economic_System_Socialism         uint8
Major_Industry_Manufacturing      uint8
Major_Industry_Services           uint8
dtype: object
                            OLS Regression Results                            
Dep. Variable:             GDP_Growth   R-squared:                       0.798
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statis

c:\users\jyb\appdata\local\programs\python\python39\lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


In [ ]:
# 결과 해석
# 1. R-squared (R²):
# 값: 0.796
# 해석: 모델이 데이터의 75.2% 변동성을 설명합니다. 
# 이는 모델이 종속 변수(GDP_Growth)의 변동성을 상당 부분 설명하고 있음을 나타냅니다.

# 2. Adj. R-squared (조정된 R²):
# 값: 0.546
# 해석: 독립 변수의 개수를 고려한 설명력으로, R-squared에 비해 낮습니다. 
# 이는 모델의 변수 수에 비해 데이터 포인트가 적어서 발생할 수 있습니다.

# 3. F-statistic:
# 값: 3.163
# Prob (F-statistic): 0.144
# 해석: 모델 전체의 유의성을 테스트하는 값입니다. p-값이 0.05보다 크므로, 모델이 통계적으로 유의하지 않습니다.

# 4. 각 변수의 회귀 계수 (coef):
# const (절편): 4.0417
# 해석: 모든 독립 변수가 0일 때의 GDP 성장률입니다.
# Region_Asia: -0.4625
# 해석: 아프리카 지역에 비해 아시아 지역일 때 GDP 성장률이 0.46% 감소합니다.
# Region_Europe: -1.1015
# 해석: 아프리카 지역에 비해 유럽 지역일 때 GDP 성장률이 % 감소합니다.

# Economic_System_Socialism: -0.1775
# 해석: 자본주의에 비해 사회주의 경제 체제일 때 GDP 성장률이 0.17% 감소합니다.
# Major_Industry_Manufacturing: -0.2333
# 해석: 농업에 비해 제조업일 때 GDP 성장률이 0.23% 감소합니다.
# Major_Industry_Services: -0.0142
# 해석: 농업에 비해 서비스업일 때 GDP 성장률이 0.014% 감소합니다.

# 5. p-값 (P>|t|):
# # Region_Europe: 0.027  통계적 유의

# 나머지 변수 :  독립 변수의 p-값이 0.05보다 큽니다. 이는 해당 변수들이 통계적으로 유의미하지 않다는 것을 의미합니다. 


# 6. 신뢰 구간 (Confidence Interval):
# 각 회귀 계수에 대한 95% 신뢰 구간은 다음과 같습니다:

# 종합 해석
# 모델 설명력: R-squared 값이 0.796로 모델의 설명력은 높습니다. 
# 조정된 R-squared 값이 낮아, 변수 수에 비해 데이터 수가 적음을 나타냅니다.
# 모델 유의성: 전체 모델과 대부분 변수 모두 통계적으로 유의하지 않습니다. 이는 데이터 수가 적거나 독립 변수가 종속 변수에 큰 영향을 미치지 않는다는 것을 의미할 수 있습니다.
# 추가 데이터 및 변수 필요: 모델의 신뢰성을 높이기 위해 더 많은 데이터 포인트를 추가하고, 다른 잠재적으로 중요한 변수를 고려하는 것이 좋습니다.
# 모델 개선을 위해 데이터 포인트를 더 많이 수집하거나, 경제 성장률에 영향을 미치는 다른 중요한 변수를 추가하는 것이 필요할 수 있습니다.